# Two-Stage Clinical Summarization Pipeline: MedGemma → Llama Compression

**Course Project - Stage 2: Summary Compression with Entity Retention**

## Pipeline Architecture

```
Clinical Notes → [Stage 1: MedGemma-4B] → Verbose Summary → [Stage 2: Llama-3-8B] → Compressed Summary
```

**Critical Design Decisions**:
- **Memory Safety**: Each model is fully unloaded before loading the next (single GPU constraint)
- **4-bit Quantization**: Both models use bitsandbytes NF4 for VRAM efficiency
- **Entity Preservation**: Chain-of-Density inspired compression with medical entity retention
- **Clinical Evaluation**: NER-based entity recall + Clinical BERTScore

**Execution Environment**: 
- Local: Runs with local file paths
- Google Colab: Integrates with Google Drive for data I/O

## Section 0: Environment Detection and Setup

Automatically detect execution environment (Colab vs Local) and configure accordingly.

In [ ]:
import os
# Detect execution environment
import sys

IS_COLAB = 'google.colab' in sys.modules

print(f"Execution Environment: {'Google Colab' if IS_COLAB else 'Local'}")

if IS_COLAB:
    print("\n⚙️  Colab-specific setup will be activated")
    print("   - Google Drive mounting")
    print("   - Drive-based data I/O")
    print("   - GPU verification")
else:
    print("\n⚙️  Local execution mode")
    print("   - Using local file paths")
    print("   - Saving outputs to project directory")

## Section 1: Installation and Imports

**Memory-Critical Libraries**:
- `bitsandbytes`: 4-bit quantization
- `accelerate`: Device mapping and offloading
- `scispacy`: Medical NER for entity extraction

In [ ]:
# Install required packages (Colab only - local assumes dependencies are installed)
if IS_COLAB:
    print("Installing dependencies for Colab...\n")
    !pip install -q transformers accelerate bitsandbytes torch
    !pip install -q scispacy
    !pip install -q https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_sm-0.5.4.tar.gz
    !pip install -q bert-score
    !pip install -q pandas numpy
    print("✓ All dependencies installed")
else:
    print("Local mode: Assuming all dependencies are already installed via uv/pip")
    print("Required: transformers, accelerate, bitsandbytes, torch, scispacy, bert-score")

In [ ]:
# Core imports
import gc
import json
import warnings

import numpy as np
import pandas as pd
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
from bert_score import BERTScorer
import spacy

warnings.filterwarnings('ignore')

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  WARNING: No GPU detected. This pipeline requires CUDA.")

## Section 2: Google Drive Setup (Colab Only)

Mount Google Drive and configure I/O paths for Colab execution.

In [ ]:
if IS_COLAB:
    from google.colab import drive

    # Mount Google Drive
    drive.mount('/content/drive')

    # Configure paths - UPDATE THESE TO MATCH YOUR DRIVE STRUCTURE
    DRIVE_BASE = "/content/drive/MyDrive/Clinical_Summarization_Project"

    # Input: Path to your MIMIC cleaned dataset
    INPUT_DATA_PATH = f"{DRIVE_BASE}/mimic_cleaned_text_only.csv"

    # Input: Path to your fine-tuned MedGemma adapters (if using PEFT)
    MEDGEMMA_ADAPTER_PATH = f"{DRIVE_BASE}/medgemma-discharge-summarization/final"

    # Output: Where to save results
    OUTPUT_DIR = f"{DRIVE_BASE}/compression_results"

    # Create output directory if it doesn't exist
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    print(f"✓ Google Drive mounted")
    print(f"\nConfigured paths:")
    print(f"  Input data: {INPUT_DATA_PATH}")
    print(f"  MedGemma adapters: {MEDGEMMA_ADAPTER_PATH}")
    print(f"  Output directory: {OUTPUT_DIR}")

    # Verify input file exists
    if os.path.exists(INPUT_DATA_PATH):
        print(f"\n✓ Input data file found")
    else:
        print(f"\n⚠️  WARNING: Input file not found at {INPUT_DATA_PATH}")
        print(f"   Please update INPUT_DATA_PATH variable above")

else:
    # Local paths
    INPUT_DATA_PATH = "mimic_cleaned_text_only.csv"
    MEDGEMMA_ADAPTER_PATH = "./medgemma-discharge-summarization/final"
    OUTPUT_DIR = "./compression_results"

    os.makedirs(OUTPUT_DIR, exist_ok=True)

    print(f"Local paths configured:")
    print(f"  Input data: {INPUT_DATA_PATH}")
    print(f"  MedGemma adapters: {MEDGEMMA_ADAPTER_PATH}")
    print(f"  Output directory: {OUTPUT_DIR}")

## Section 3: Configuration and Hyperparameters

**Critical Memory Settings**:
- Both models use NF4 4-bit quantization
- `device_map="auto"` for optimal GPU distribution
- Explicit memory flushing between stages

In [ ]:
# ============================================================================
# MODEL IDENTIFIERS
# ============================================================================

# Stage 1: High-recall summary generation
MEDGEMMA_BASE_MODEL = "google/medgemma-4b-it"
USE_MEDGEMMA_ADAPTER = True  # Set to False if using base model without LoRA

# Stage 2: Compression model
LLAMA_MODEL = "meta-llama/Meta-Llama-3-8B-Instruct"

# ============================================================================
# QUANTIZATION CONFIG (Shared across both models)
# ============================================================================

BNB_CONFIG = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

# ============================================================================
# GENERATION PARAMETERS
# ============================================================================

# Stage 1: Verbose summary generation
STAGE1_MAX_TOKENS = 512
STAGE1_TEMPERATURE = 0.7
STAGE1_TOP_P = 0.9

# Stage 2: Compression generation
STAGE2_MAX_TOKENS = 256  # Deliberately shorter for compression
STAGE2_TEMPERATURE = 0.3  # Lower temperature for more deterministic compression
STAGE2_TOP_P = 0.9

# ============================================================================
# EVALUATION SETTINGS
# ============================================================================

# Number of samples to process (set lower for quick testing)
NUM_SAMPLES = 10  # Change to -1 to process entire dataset

# NER model for entity extraction
SPACY_MODEL = "en_core_sci_sm"  # Medical NER model

# Clinical BERTScore model
CLINICAL_BERT = "emilyalsentzer/Bio_ClinicalBERT"

print("✓ Configuration loaded")
print(f"\nStage 1 Model: {MEDGEMMA_BASE_MODEL}")
print(f"  - Using LoRA adapters: {USE_MEDGEMMA_ADAPTER}")
print(f"  - Max tokens: {STAGE1_MAX_TOKENS}")
print(f"\nStage 2 Model: {LLAMA_MODEL}")
print(f"  - Max tokens: {STAGE2_MAX_TOKENS}")
print(f"  - Temperature: {STAGE2_TEMPERATURE} (deterministic compression)")
print(f"\nProcessing {NUM_SAMPLES if NUM_SAMPLES > 0 else 'ALL'} samples")

## Section 4: Memory Management Utilities

**Critical for Single-GPU Execution**: These functions ensure complete model unloading between stages.

In [ ]:
def flush_memory():
    """
    Aggressively flush GPU and CPU memory.
    
    This is CRITICAL when switching between models on a single GPU.
    Without this, you will encounter OOM errors.
    """
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

    # Report memory status
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated(0) / 1e9
        reserved = torch.cuda.memory_reserved(0) / 1e9
        print(f"GPU Memory: {allocated:.2f} GB allocated, {reserved:.2f} GB reserved")


def unload_model(model, tokenizer):
    """
    Completely unload a model and tokenizer from memory.
    
    Args:
        model: The model to unload (or None)
        tokenizer: The tokenizer to unload (or None)
    """
    print("\n🧹 Unloading model from memory...")

    if model is not None:
        # Move model to CPU first (if on GPU)
        if hasattr(model, 'cpu'):
            model.cpu()
        del model

    if tokenizer is not None:
        del tokenizer

    flush_memory()
    print("✓ Model unloaded\n")


# Test memory utilities
print("Memory management utilities loaded\n")
flush_memory()

## Section 5: Load Input Data

Load clinical notes dataset (preprocessed from Stage 0).

In [ ]:
print(f"Loading data from: {INPUT_DATA_PATH}\n")

# Load dataset
df = pd.read_csv(INPUT_DATA_PATH)

print(f"✓ Dataset loaded: {len(df)} total samples")
print(f"  Columns: {list(df.columns)}")

# Select subset if configured
if NUM_SAMPLES > 0 and NUM_SAMPLES < len(df):
    df = df.head(NUM_SAMPLES)
    print(f"\n📊 Processing {len(df)} samples (subset for testing)")
else:
    print(f"\n📊 Processing all {len(df)} samples")

# Display sample
print(f"\n{'=' * 80}")
print("SAMPLE INPUT (First 300 chars):")
print(f"{'=' * 80}")
print(df.iloc[0]['final_input'][:300] + "...")
print(f"\n{'=' * 80}")

## Section 6: STAGE 1 - MedGemma Verbose Summary Generation

**Objective**: Generate high-recall, detailed clinical summaries.

**Memory Strategy**: 
1. Load MedGemma-4B in 4-bit
2. Generate summaries for all samples
3. **CRITICAL**: Completely unload before Stage 2

In [ ]:
print("=" * 80)
print("STAGE 1: LOADING MEDGEMMA-4B FOR VERBOSE SUMMARY GENERATION")
print("=" * 80)

# Load tokenizer
print(f"\nLoading tokenizer: {MEDGEMMA_BASE_MODEL}")
medgemma_tokenizer = AutoTokenizer.from_pretrained(
    MEDGEMMA_BASE_MODEL,
    trust_remote_code=True,
    padding_side="right",
    add_eos_token=True
)
medgemma_tokenizer.pad_token = medgemma_tokenizer.eos_token
print("✓ Tokenizer loaded")

# Load base model with quantization
print(f"\nLoading model: {MEDGEMMA_BASE_MODEL}")
print("  Quantization: 4-bit NF4")
print("  This may take 2-3 minutes...")

medgemma_model = AutoModelForCausalLM.from_pretrained(
    MEDGEMMA_BASE_MODEL,
    quantization_config=BNB_CONFIG,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)

print("✓ Base model loaded")

# Load LoRA adapters if configured
if USE_MEDGEMMA_ADAPTER and os.path.exists(MEDGEMMA_ADAPTER_PATH):
    print(f"\nLoading LoRA adapters from: {MEDGEMMA_ADAPTER_PATH}")
    from peft import PeftModel

    medgemma_model = PeftModel.from_pretrained(medgemma_model, MEDGEMMA_ADAPTER_PATH)
    print("✓ LoRA adapters loaded")
elif USE_MEDGEMMA_ADAPTER:
    print(f"\n⚠️  WARNING: Adapter path not found: {MEDGEMMA_ADAPTER_PATH}")
    print("   Continuing with base model only")

medgemma_model.eval()
print(f"\n✓ MedGemma ready for inference")
flush_memory()

In [ ]:
def generate_stage1_summary(clinical_note: str) -> str:
    """
    Generate verbose clinical summary using MedGemma.
    
    Args:
        clinical_note: Input clinical text
    
    Returns:
        Generated summary (high-recall, verbose)
    """
    instruction = (
        "Summarize the following clinical discharge notes. "
        "Include ALL diagnoses, medications, vitals, lab results, "
        "procedures, and follow-up instructions. "
        "Ensure complete coverage of all medical entities."
    )

    # Format using Gemma template
    prompt = f"""<start_of_turn>user
{instruction}

Clinical Notes:
{clinical_note}<end_of_turn>
<start_of_turn>model
"""

    inputs = medgemma_tokenizer(prompt, return_tensors="pt").to(medgemma_model.device)

    with torch.no_grad():
        outputs = medgemma_model.generate(
            **inputs,
            max_new_tokens=STAGE1_MAX_TOKENS,
            temperature=STAGE1_TEMPERATURE,
            top_p=STAGE1_TOP_P,
            do_sample=True,
            pad_token_id=medgemma_tokenizer.pad_token_id,
            eos_token_id=medgemma_tokenizer.eos_token_id
        )

    generated_text = medgemma_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the model's response
    marker = "<start_of_turn>model"
    if marker in generated_text:
        summary = generated_text.split(marker)[-1].strip()
    else:
        summary = generated_text[len(prompt):].strip()

    return summary


print("✓ Stage 1 generation function defined")

In [ ]:
# Generate Stage 1 summaries for all samples
print(f"\nGenerating Stage 1 summaries for {len(df)} samples...\n")

stage1_summaries = []

for idx, row in df.iterrows():
    clinical_note = row['final_input']

    print(f"[{idx + 1}/{len(df)}] Generating verbose summary...", end=" ")
    summary = generate_stage1_summary(clinical_note)
    stage1_summaries.append(summary)
    print(f"✓ ({len(summary)} chars)")

# Add to dataframe
df['stage1_summary'] = stage1_summaries

print(f"\n✓ Stage 1 complete: {len(stage1_summaries)} summaries generated")
print(f"  Average length: {np.mean([len(s) for s in stage1_summaries]):.0f} characters")

# Display sample
print(f"\n{'=' * 80}")
print("STAGE 1 OUTPUT SAMPLE:")
print(f"{'=' * 80}")
print(stage1_summaries[0][:400] + "...")
print(f"{'=' * 80}")

### CRITICAL: Unload MedGemma Before Stage 2

**This step is NON-NEGOTIABLE for single-GPU execution.**

In [ ]:
print("\n" + "=" * 80)
print("UNLOADING MEDGEMMA-4B FROM MEMORY")
print("=" * 80)

unload_model(medgemma_model, medgemma_tokenizer)

# Verify memory is cleared
print("Memory status after unloading:")
flush_memory()

print("\n✓ Safe to proceed to Stage 2")

## Section 7: STAGE 2 - Llama-3 Compression

**Objective**: Compress verbose summaries while preserving medical entities.

**Chain-of-Density Inspired Prompt**: Forces 50% compression with entity retention constraint.

In [ ]:
print("=" * 80)
print("STAGE 2: LOADING LLAMA-3-8B FOR COMPRESSION")
print("=" * 80)

# Check if we need HuggingFace authentication for Llama
if IS_COLAB:
    print("\n⚠️  Llama-3 requires HuggingFace authentication")
    print("   Running authentication flow...\n")
    from huggingface_hub import notebook_login

    notebook_login()

# Load tokenizer
print(f"\nLoading tokenizer: {LLAMA_MODEL}")
llama_tokenizer = AutoTokenizer.from_pretrained(
    LLAMA_MODEL,
    trust_remote_code=True,
    padding_side="left"  # Llama uses left padding
)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
print("✓ Tokenizer loaded")

# Load model with quantization
print(f"\nLoading model: {LLAMA_MODEL}")
print("  Quantization: 4-bit NF4")
print("  This may take 2-3 minutes...")

llama_model = AutoModelForCausalLM.from_pretrained(
    LLAMA_MODEL,
    quantization_config=BNB_CONFIG,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)

llama_model.eval()
print(f"\n✓ Llama-3 ready for compression")
flush_memory()

In [ ]:
def generate_compressed_summary(verbose_summary: str) -> str:
    """
    Compress summary using Llama-3 with entity retention constraint.
    
    Chain-of-Density inspired prompt: Forces 50% compression while
    preserving all medical entities (medications, numbers, dates).
    
    Args:
        verbose_summary: Stage 1 summary (high-recall)
    
    Returns:
        Compressed summary
    """
    # Chain-of-Density inspired compression prompt
    system_prompt = (
        "You are a medical summarization expert. Your task is to compress "
        "clinical summaries while preserving critical information."
    )

    user_prompt = (
        "Rewrite the following summary to be 50% shorter. "
        "You MUST retain ALL entities: medications (with dosages), "
        "vital signs (with numbers), lab results (with values), "
        "diagnoses, procedures, and dates. "
        "If you cannot shorten it without losing a critical fact, do not shorten it. "
        "Remove only redundant phrasing and verbose descriptions.\n\n"
        f"Summary to compress:\n{verbose_summary}"
    )

    # Format using Llama-3 chat template
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    prompt = llama_tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = llama_tokenizer(prompt, return_tensors="pt").to(llama_model.device)

    with torch.no_grad():
        outputs = llama_model.generate(
            **inputs,
            max_new_tokens=STAGE2_MAX_TOKENS,
            temperature=STAGE2_TEMPERATURE,
            top_p=STAGE2_TOP_P,
            do_sample=True,
            pad_token_id=llama_tokenizer.pad_token_id,
            eos_token_id=llama_tokenizer.eos_token_id
        )

    generated_text = llama_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the assistant's response
    # Llama-3 format: <|start_header_id|>assistant<|end_header_id|>\n\n{response}
    if "assistant" in generated_text:
        compressed = generated_text.split("assistant")[-1].strip()
        # Remove any remaining header artifacts
        compressed = compressed.split("\n\n", 1)[-1] if "\n\n" in compressed else compressed
    else:
        compressed = generated_text[len(prompt):].strip()

    return compressed


print("✓ Stage 2 compression function defined")

In [ ]:
# Generate Stage 2 compressed summaries
print(f"\nCompressing {len(df)} Stage 1 summaries...\n")

stage2_summaries = []

for idx, row in df.iterrows():
    verbose_summary = row['stage1_summary']

    print(f"[{idx + 1}/{len(df)}] Compressing summary...", end=" ")
    compressed = generate_compressed_summary(verbose_summary)
    stage2_summaries.append(compressed)

    # Calculate compression ratio
    ratio = len(compressed) / len(verbose_summary)
    print(f"✓ ({len(compressed)} chars, {ratio:.2%} of original)")

# Add to dataframe
df['stage2_summary'] = stage2_summaries

print(f"\n✓ Stage 2 complete: {len(stage2_summaries)} summaries compressed")
print(f"  Average length: {np.mean([len(s) for s in stage2_summaries]):.0f} characters")
print(
    f"  Overall compression: {np.mean([len(s2) / len(s1) for s1, s2 in zip(stage1_summaries, stage2_summaries)]):.2%}")

# Display sample
print(f"\n{'=' * 80}")
print("STAGE 2 OUTPUT SAMPLE:")
print(f"{'=' * 80}")
print(stage2_summaries[0])
print(f"{'=' * 80}")

### Unload Llama-3 (Optional - only if running evaluation separately)

In [ ]:
# If you need to free memory before evaluation, uncomment:
# unload_model(llama_model, llama_tokenizer)

print("✓ Stage 2 generation complete. Proceeding to evaluation...")

## Section 8: EVALUATION - Metric 1: Compression Ratio

In [ ]:
print("=" * 80)
print("EVALUATION METRIC 1: COMPRESSION RATIO")
print("=" * 80)


def calculate_compression_ratio(stage1: str, stage2: str) -> float:
    """
    Calculate compression ratio (Stage 2 length / Stage 1 length).
    
    Lower values indicate stronger compression.
    """
    return len(stage2) / len(stage1)


# Calculate for all samples
df['compression_ratio'] = df.apply(
    lambda row: calculate_compression_ratio(row['stage1_summary'], row['stage2_summary']),
    axis=1
)

print(f"\nCompression Statistics:")
print(f"  Mean ratio: {df['compression_ratio'].mean():.2%}")
print(f"  Median ratio: {df['compression_ratio'].median():.2%}")
print(f"  Min ratio: {df['compression_ratio'].min():.2%}")
print(f"  Max ratio: {df['compression_ratio'].max():.2%}")
print(f"  Std dev: {df['compression_ratio'].std():.2%}")

print(f"\n✓ Compression ratios calculated")

## Section 9: EVALUATION - Metric 2: Entity Retention (NER)

**Method**: Extract medical entities from both Stage 1 and Stage 2 summaries using SciSpacy, then calculate recall.

In [ ]:
print("=" * 80)
print("EVALUATION METRIC 2: ENTITY RETENTION (NER)")
print("=" * 80)

# Load SciSpacy medical NER model
print(f"\nLoading SciSpacy model: {SPACY_MODEL}")
nlp = spacy.load(SPACY_MODEL)
print("✓ NER model loaded")


def extract_entities(text: str) -> set:
    """
    Extract medical entities from text using SciSpacy.
    
    Returns set of entity texts (lowercased for comparison).
    """
    doc = nlp(text)
    entities = {ent.text.lower() for ent in doc.ents}
    return entities


def calculate_entity_recall(stage1_text: str, stage2_text: str) -> float:
    """
    Calculate entity retention recall: 
    (Entities in Stage 2) / (Entities in Stage 1)
    
    Returns value between 0 and 1 (higher is better).
    """
    entities_stage1 = extract_entities(stage1_text)
    entities_stage2 = extract_entities(stage2_text)

    if len(entities_stage1) == 0:
        return 1.0  # No entities to preserve

    # Calculate recall: how many Stage 1 entities appear in Stage 2?
    retained = entities_stage1.intersection(entities_stage2)
    recall = len(retained) / len(entities_stage1)

    return recall


print("\nCalculating entity recall for all samples...\n")

entity_recalls = []
for idx, row in df.iterrows():
    print(f"[{idx + 1}/{len(df)}] Extracting entities...", end=" ")
    recall = calculate_entity_recall(row['stage1_summary'], row['stage2_summary'])
    entity_recalls.append(recall)
    print(f"✓ Recall: {recall:.2%}")

df['entity_recall'] = entity_recalls

print(f"\nEntity Retention Statistics:")
print(f"  Mean recall: {df['entity_recall'].mean():.2%}")
print(f"  Median recall: {df['entity_recall'].median():.2%}")
print(f"  Min recall: {df['entity_recall'].min():.2%}")
print(f"  Max recall: {df['entity_recall'].max():.2%}")
print(f"  Std dev: {df['entity_recall'].std():.2%}")

print(f"\n✓ Entity recall calculated")

## Section 10: EVALUATION - Metric 3: Clinical BERTScore

**Method**: Measure semantic similarity between original clinical notes and final compressed summaries using Bio_ClinicalBERT.

In [ ]:
print("=" * 80)
print("EVALUATION METRIC 3: CLINICAL BERTSCORE")
print("=" * 80)

# Initialize Clinical BERTScore
print(f"\nInitializing BERTScorer with {CLINICAL_BERT}")
print("This will download the model if not cached...\n")

bert_scorer = BERTScorer(
    model_type=CLINICAL_BERT,
    num_layers=9,
    rescale_with_baseline=True,
    lang="en",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

print("✓ BERTScorer initialized")

print("\nCalculating BERTScores (this may take a few minutes)...\n")

# Prepare data for batch scoring
source_texts = df['final_input'].tolist()
compressed_summaries = df['stage2_summary'].tolist()

# Calculate BERTScore
# We compare compressed summaries (candidates) against original notes (references)
P, R, F1 = bert_scorer.score(
    cands=compressed_summaries,
    refs=source_texts
)

# Convert to numpy
df['bertscore_precision'] = P.cpu().numpy()
df['bertscore_recall'] = R.cpu().numpy()
df['bertscore_f1'] = F1.cpu().numpy()

print(f"\nClinical BERTScore Statistics:")
print(f"\nPrecision:")
print(f"  Mean: {df['bertscore_precision'].mean():.4f}")
print(f"  Median: {df['bertscore_precision'].median():.4f}")
print(f"\nRecall:")
print(f"  Mean: {df['bertscore_recall'].mean():.4f}")
print(f"  Median: {df['bertscore_recall'].median():.4f}")
print(f"\nF1:")
print(f"  Mean: {df['bertscore_f1'].mean():.4f}")
print(f"  Median: {df['bertscore_f1'].median():.4f}")

print(f"\n✓ Clinical BERTScore calculated")

## Section 11: Results Summary and Export

In [ ]:
print("=" * 80)
print("FINAL RESULTS SUMMARY")
print("=" * 80)

# Select columns for final output
results_df = df[[
    'final_input',
    'stage1_summary',
    'stage2_summary',
    'compression_ratio',
    'entity_recall',
    'bertscore_precision',
    'bertscore_recall',
    'bertscore_f1'
]].copy()

# Rename for clarity
results_df.columns = [
    'Source_Text',
    'Stage1_Summary_Verbose',
    'Stage2_Summary_Compressed',
    'Compression_Ratio',
    'Entity_Recall',
    'BERTScore_Precision',
    'BERTScore_Recall',
    'BERTScore_F1'
]

# Display summary statistics
print(f"\nProcessed {len(results_df)} samples\n")
print("Aggregate Metrics:")
print(
    f"  Compression Ratio: {results_df['Compression_Ratio'].mean():.2%} ± {results_df['Compression_Ratio'].std():.2%}")
print(f"  Entity Recall: {results_df['Entity_Recall'].mean():.2%} ± {results_df['Entity_Recall'].std():.2%}")
print(f"  BERTScore F1: {results_df['BERTScore_F1'].mean():.4f} ± {results_df['BERTScore_F1'].std():.4f}")

# Display sample comparison
print(f"\n{'=' * 80}")
print("SAMPLE COMPARISON (First Example)")
print(f"{'=' * 80}")
sample = results_df.iloc[0]
print(f"\nSource Text (truncated): {sample['Source_Text'][:200]}...")
print(f"\nStage 1 (Verbose): {sample['Stage1_Summary_Verbose'][:300]}...")
print(f"\nStage 2 (Compressed): {sample['Stage2_Summary_Compressed']}")
print(f"\nMetrics:")
print(f"  Compression: {sample['Compression_Ratio']:.2%}")
print(f"  Entity Recall: {sample['Entity_Recall']:.2%}")
print(f"  BERTScore F1: {sample['BERTScore_F1']:.4f}")
print(f"{'=' * 80}")

In [ ]:
# Save results
output_csv_path = os.path.join(OUTPUT_DIR, "compression_pipeline_results.csv")
output_json_path = os.path.join(OUTPUT_DIR, "compression_pipeline_results.json")

# Save as CSV
results_df.to_csv(output_csv_path, index=False)
print(f"\n✓ Results saved to CSV: {output_csv_path}")

# Save as JSON (for easier inspection)
results_df.to_json(output_json_path, orient='records', indent=2)
print(f"✓ Results saved to JSON: {output_json_path}")

# Save summary statistics
summary_stats = {
    "num_samples": len(results_df),
    "compression_ratio": {
        "mean": float(results_df['Compression_Ratio'].mean()),
        "std": float(results_df['Compression_Ratio'].std()),
        "min": float(results_df['Compression_Ratio'].min()),
        "max": float(results_df['Compression_Ratio'].max())
    },
    "entity_recall": {
        "mean": float(results_df['Entity_Recall'].mean()),
        "std": float(results_df['Entity_Recall'].std()),
        "min": float(results_df['Entity_Recall'].min()),
        "max": float(results_df['Entity_Recall'].max())
    },
    "bertscore_f1": {
        "mean": float(results_df['BERTScore_F1'].mean()),
        "std": float(results_df['BERTScore_F1'].std()),
        "min": float(results_df['BERTScore_F1'].min()),
        "max": float(results_df['BERTScore_F1'].max())
    },
    "config": {
        "stage1_model": MEDGEMMA_BASE_MODEL,
        "stage2_model": LLAMA_MODEL,
        "num_samples_processed": NUM_SAMPLES if NUM_SAMPLES > 0 else "all"
    }
}

summary_path = os.path.join(OUTPUT_DIR, "summary_statistics.json")
with open(summary_path, 'w') as f:
    json.dump(summary_stats, f, indent=2)

print(f"✓ Summary statistics saved: {summary_path}")

if IS_COLAB:
    print(f"\n📁 All outputs saved to Google Drive: {OUTPUT_DIR}")
else:
    print(f"\n📁 All outputs saved to: {OUTPUT_DIR}")

print("\n" + "=" * 80)
print("PIPELINE COMPLETE")
print("=" * 80)

## Section 12: Analysis and Insights

Key questions to investigate:
1. **Compression vs Quality Trade-off**: Does higher compression correlate with lower entity recall?
2. **BERTScore Reliability**: Does Clinical BERTScore align with entity retention?
3. **Failure Modes**: Which samples fail to compress or lose critical entities?

In [ ]:
# Analysis 1: Correlation between compression and entity recall
import seaborn as sns

if len(results_df) > 1:
    correlation = results_df['Compression_Ratio'].corr(results_df['Entity_Recall'])
    print(f"Correlation (Compression Ratio vs Entity Recall): {correlation:.3f}")

    if correlation < -0.3:
        print("  → Strong negative correlation: Higher compression → Lower entity retention")
    elif correlation > 0.3:
        print("  → Strong positive correlation: Counterintuitive result, investigate further")
    else:
        print("  → Weak correlation: Compression and entity retention are relatively independent")

# Analysis 2: Identify best and worst performing samples
print("\n" + "=" * 80)
print("Best Performing Sample (Highest Entity Recall):")
print("=" * 80)
best_idx = results_df['Entity_Recall'].idxmax()
best = results_df.iloc[best_idx]
print(f"Entity Recall: {best['Entity_Recall']:.2%}")
print(f"Compression: {best['Compression_Ratio']:.2%}")
print(f"BERTScore F1: {best['BERTScore_F1']:.4f}")
print(f"\nStage 2 Summary: {best['Stage2_Summary_Compressed'][:300]}...")

print("\n" + "=" * 80)
print("Worst Performing Sample (Lowest Entity Recall):")
print("=" * 80)
worst_idx = results_df['Entity_Recall'].idxmin()
worst = results_df.iloc[worst_idx]
print(f"Entity Recall: {worst['Entity_Recall']:.2%}")
print(f"Compression: {worst['Compression_Ratio']:.2%}")
print(f"BERTScore F1: {worst['BERTScore_F1']:.4f}")
print(f"\nStage 2 Summary: {worst['Stage2_Summary_Compressed'][:300]}...")

print("\n✓ Analysis complete")

## Final Notes and Next Steps

### Memory Management Recap
This notebook demonstrated:
- ✅ Loading 4B and 8B models sequentially on a single GPU
- ✅ Complete model unloading with `flush_memory()`
- ✅ Successful execution without OOM errors

### Results Interpretation
- **Compression Ratio**: Target was ~50%, actual performance depends on prompt adherence
- **Entity Recall**: Should be >85% for production use
- **BERTScore**: Higher F1 indicates better semantic preservation

### Potential Improvements
1. **Prompt Engineering**: Refine compression prompt for better entity retention
2. **Multi-stage Compression**: Iterative compression with entity verification
3. **Entity-Aware Loss**: Fine-tune Llama on compression with entity-weighted loss
4. **Hybrid Approach**: Extractive + abstractive compression